### Attempting to run regressions from systematics values extracted geometrically or from Kitanidis et al on galaxy density


In [68]:
import pandas as pd
import numpy as np
import pickle
import healpy as hp

In [69]:
### Setting Healpy Information
NSIDE = 256
NPIX = hp.nside2npix(NSIDE)
print(
    "Approximate resolution at NSIDE {} is {:.2} deg".format(
        NSIDE, hp.nside2resol(NSIDE, arcmin=True) / 60
    )
)

def raDec2thetaPhi(ra, dec):
    return (0.5 * np.pi - np.deg2rad(dec)), (np.deg2rad(ra))


Approximate resolution at NSIDE 256 is 0.23 deg


### Importing Galaxy Density Information

In [70]:
df_objects = pd.read_csv('../bricks_data/galaxy_catalogue_sample.csv')


In [71]:
df_objects.head()

,BrickID,ObjectID,RA,DEC,South,Target_type
0,389675,10,133.284363,10.315927,True,3
1,389675,11,133.284832,10.314327,True,2
2,389675,15,133.285109,10.294865,True,3
3,389675,21,133.285731,10.319199,True,3
4,389675,35,133.286350,10.168023,True,2


In [72]:
# LRG

df_LRG = df_objects[df_objects["Target_type"] == 1]
ra_LRG = df_LRG["RA"].to_numpy(copy=True)
dec_LRG = df_LRG["DEC"].to_numpy(copy=True)
theta, phi = raDec2thetaPhi(ra_LRG, dec_LRG)

print("Number of LRGs in Sample:", len(ra_LRG))

LRG_pixel_indices = hp.ang2pix(NSIDE, theta,phi)


# Finding out unique indices and how often they appear --> shows the density of LRGs in this pixel
(unique, counts) = np.unique(LRG_pixel_indices, return_counts=True)

print(unique)

mean_LRG = counts.mean()
print("Mean LRGs per Pixel:", mean_LRG)
# Calculating Densities for every block
density_LRG = (counts / mean_LRG) - 1
id_density = np.stack((unique, density_LRG), axis=1)
print(id_density)
df_lrg = pd.DataFrame(id_density, columns=['pixel_id', 'lrg_density'])
df_lrg.pixel_id = df_lrg.pixel_id.astype(int)
df_lrg.head()

Number of LRGs in Sample: 41904
[166098 167122 167123 ... 756988 756989 757472]
Mean LRGs per Pixel: 9.93692198245198
[[ 1.66098000e+05  3.08252195e-01]
 [ 1.67122000e+05 -8.99365216e-01]
 [ 1.67123000e+05  4.08886980e-01]
 ...
 [ 7.56988000e+05 -8.99365216e-01]
 [ 7.56989000e+05  1.61650439e+00]
 [ 7.57472000e+05 -4.96826079e-01]]


,pixel_id,lrg_density
0,166098,0.308252
1,167122,-0.899365
2,167123,0.408887
3,171167,-0.496826
4,172190,-0.496826


In [73]:
# LRG

df_ELG = df_objects[df_objects["Target_type"] == 2]
ra_ELG = df_ELG["RA"].to_numpy(copy=True)
dec_ELG = df_ELG["DEC"].to_numpy(copy=True)
theta, phi = raDec2thetaPhi(ra_ELG, dec_ELG)

print("Number of ELGs in Sample:", len(ra_ELG))

ELG_pixel_indices = hp.ang2pix(NSIDE, theta,phi)


# Finding out unique indices and how often they appear --> shows the density of LRGs in this pixel
(unique, counts) = np.unique(ELG_pixel_indices, return_counts=True)

print(unique)

mean_ELG = counts.mean()
print("Mean ELGs per Pixel:", mean_ELG)
# Calculating Densities for every block
density_ELG = (counts / mean_ELG) - 1
id_density = np.stack((unique, density_ELG), axis=1)
print(id_density)
df_elg = pd.DataFrame(id_density, columns=['pixel_id', 'elg_density'])
df_elg.pixel_id = df_elg.pixel_id.astype(int)

print(df_elg.head())

Number of ELGs in Sample: 157118
[166098 167122 167123 ... 756502 756989 757472]
Mean ELGs per Pixel: 32.556568586821385
[[ 1.66098000e+05  7.50518718e-02]
 [ 1.67122000e+05 -9.07852697e-01]
 [ 1.67123000e+05 -4.16400412e-01]
 ...
 [ 7.56502000e+05 -5.69979251e-01]
 [ 7.56989000e+05  4.34454359e+00]
 [ 7.57472000e+05  1.36203363e-02]]
   pixel_id  elg_density
0    166098     0.075052
1    167122    -0.907853
2    167123    -0.416400
3    171167     0.412925
4    172190    -0.293537


In [74]:
df_QSO = df_objects[df_objects["Target_type"] == 3]
ra_QSO = df_QSO["RA"].to_numpy(copy=True)
dec_QSO = df_QSO["DEC"].to_numpy(copy=True)
theta, phi = raDec2thetaPhi(ra_QSO, dec_QSO)

print("Number of QSOs in Sample:", len(ra_QSO))

QSO_pixel_indices = hp.ang2pix(NSIDE, theta,phi)


# Finding out unique indices and how often they appear --> shows the density of LRGs in this pixel
(unique, counts) = np.unique(QSO_pixel_indices, return_counts=True)

print(unique)

mean_QSO = counts.mean()
print("Mean QSOs per Pixel:", mean_QSO)
# Calculating Densities for every block
density_QSO = (counts / mean_QSO) - 1
id_density = np.stack((unique, density_QSO), axis=1)
print(id_density)
df_qso = pd.DataFrame(id_density, columns=['pixel_id', 'qso_density'])
df_qso.pixel_id = df_qso.pixel_id.astype(int)

print(df_qso.head())

Number of QSOs in Sample: 180394
[166098 167122 167123 ... 756502 756989 757472]
Mean QSOs per Pixel: 36.92058943921408
[[ 1.66098000e+05  5.63211637e-02]
 [ 1.67122000e+05 -8.10403894e-01]
 [ 1.67123000e+05  1.10491480e-01]
 ...
 [ 7.56502000e+05 -8.64574210e-01]
 [ 7.56989000e+05  2.60232602e+00]
 [ 7.57472000e+05 -1.60360101e-01]]
   pixel_id  qso_density
0    166098     0.056321
1    167122    -0.810404
2    167123     0.110491
3    171167    -0.024934
4    172190    -0.133275


In [75]:
df = pd.merge(df_lrg, df_elg, how='outer', on='pixel_id')
df = pd.merge(df, df_qso, how='outer', on='pixel_id')
print(df.isna().sum())
print(df_lrg.shape, df_elg.shape, df_qso.shape, df.shape)

pixel_id         0
lrg_density    860
elg_density    251
qso_density    191
dtype: int64
(4217, 2) (4826, 2) (4886, 2) (5077, 4)


In [76]:
# Dropping all pixels where I do not have densities for all 3 types of galaxies
df.dropna(axis=0, inplace=True)
print(df.isna().sum())

pixel_id       0
lrg_density    0
elg_density    0
qso_density    0
dtype: int64


### Importing the Systematics Values and Appending them to the df.


#### Stellar Density

In [77]:
df_Stellar = pd.read_csv('../bricks_data/galaxy_catalogue_stellar.csv')
ra_stellar = df_Stellar["RA"].to_numpy(copy=True)
dec_stellar = df_Stellar["DEC"].to_numpy(copy=True)
theta, phi = raDec2thetaPhi(ra_stellar, dec_stellar)

print("Number of stellars in Sample:", len(ra_stellar))

stellar_pixel_indices = hp.ang2pix(NSIDE, theta,phi)


# Finding out unique indices and how often they appear --> shows the density of LRGs in this pixel
(unique, counts) = np.unique(stellar_pixel_indices, return_counts=True)

print(unique)

mean_stellar = counts.mean()
print("Mean stellars per Pixel:", mean_stellar)
# Calculating Densities for every block
density_stellar = (counts / mean_stellar) - 1
id_density = np.stack((unique, density_stellar), axis=1)
print(id_density)
df_stellar = pd.DataFrame(id_density, columns=['pixel_id', 'stellar_density'])
df_stellar.pixel_id = df_stellar.pixel_id.astype(int)

print(df_stellar.head())

print(df_stellar.isna().sum())

Number of stellars in Sample: 4739
[166098 167123 168146 ... 756988 756989 757472]
Mean stellars per Pixel: 2.263132760267431
[[ 1.66098000e+05  3.25596117e-01]
 [ 1.67123000e+05 -1.16269255e-01]
 [ 1.68146000e+05 -5.58134628e-01]
 ...
 [ 7.56988000e+05 -5.58134628e-01]
 [ 7.56989000e+05  1.20932686e+00]
 [ 7.57472000e+05 -5.58134628e-01]]
   pixel_id  stellar_density
0    166098         0.325596
1    167123        -0.116269
2    168146        -0.558135
3    170236         2.093058
4    171167        -0.558135
pixel_id           0
stellar_density    0
dtype: int64


In [78]:
df = df.merge(df_stellar, how='inner', on='pixel_id')
print(df.isna().sum())
df.shape

pixel_id           0
lrg_density        0
elg_density        0
qso_density        0
stellar_density    0
dtype: int64


(2002, 5)

#### Kitanidis Systematics Values

In [80]:
with open('../bricks_data/pixel2systematics_kitanidis.pickle', 'rb') as f:
    kitanidis_dict = pickle.load(f)
    f.close()

print(len(kitanidis_dict.keys()))

1567


In [88]:
pixel_keys = kitanidis_dict.keys()

sys_list = []
for key in pixel_keys:
    ind_sys_list = kitanidis_dict[key]
    ind_sys_list.append(key)
    sys_list.append(ind_sys_list)

df_sys_kit = pd.DataFrame(sys_list, columns=['airmass_galaxy',
'fwhm_galaxy',
'ebv_galaxy',
'ccdnphotom_galaxy',

'ccdskysb_galaxy_g',
'ccdskysb_galaxy_r',
'ccdskysb_galaxy_z',
'exptime_galaxy_g',
'exptime_galaxy_r',
'exptime_galaxy_z',
'meansky_galaxy_g',
'meansky_galaxy_r',
'meansky_galaxy_z',
'galdepth_galaxy_g',
'galdepth_galaxy_r',
'galdepth_galaxy_z', 'pixel_id'])



ValueError: 17 columns passed, passed data had 18 columns

In [89]:
df_sys_kit.head()

,airmass_galaxy,fwhm_galaxy,ebv_galaxy,ccdnphotom_galaxy,ccdskysb_galaxy_g,ccdskysb_galaxy_r,ccdskysb_galaxy_z,exptime_galaxy_g,exptime_galaxy_r,exptime_galaxy_z,meansky_galaxy_g,meansky_galaxy_r,meansky_galaxy_z,galdepth_galaxy_g,galdepth_galaxy_r,galdepth_galaxy_z,pixel_id
0,1.204206,4.569465,0.021082,273.100211,23.627050,22.730956,20.206251,90.0,90.0,90.0,2.390975e+08,6.489436e+08,2.384546e+09,23.111424,23.139025,22.097923,741107
1,1.069105,4.824085,0.010359,148.191320,23.601375,22.732608,20.000021,90.0,90.0,90.0,1.803126e+08,2.993781e+08,2.225294e+10,23.353823,23.320494,21.550398,662895
2,1.108757,4.413348,0.011393,154.348513,23.602864,22.701580,20.073285,90.0,90.0,90.0,2.952632e+08,8.687714e+08,1.152634e+10,23.062541,22.892199,21.675296,662913
3,1.104669,4.394738,0.011230,153.649709,23.613133,22.710532,20.111641,90.0,90.0,90.0,2.446651e+08,9.609978e+08,8.914973e+09,23.186001,22.788259,21.786543,663905
4,1.107077,4.439277,0.014093,150.531232,23.583476,22.647124,20.197871,90.0,90.0,90.0,3.645015e+08,6.979010e+08,9.072512e+09,22.836915,23.042679,21.724951,652792


In [92]:
df_kitanidis = df.merge(df_sys_kit, how='inner', on='pixel_id')
print(df_kitanidis.isna().sum())
df_kitanidis.dropna(axis=0, inplace=True)

df_kitanidis.head()


pixel_id             0
lrg_density          0
elg_density          0
qso_density          0
stellar_density      0
airmass_galaxy       0
fwhm_galaxy          0
ebv_galaxy           0
ccdnphotom_galaxy    0
ccdskysb_galaxy_g    3
ccdskysb_galaxy_r    0
ccdskysb_galaxy_z    0
exptime_galaxy_g     0
exptime_galaxy_r     0
exptime_galaxy_z     0
meansky_galaxy_g     0
meansky_galaxy_r     0
meansky_galaxy_z     0
galdepth_galaxy_g    0
galdepth_galaxy_r    0
galdepth_galaxy_z    0
dtype: int64


,pixel_id,lrg_density,elg_density,qso_density,stellar_density,airmass_galaxy,fwhm_galaxy,ebv_galaxy,ccdnphotom_galaxy,ccdskysb_galaxy_g,...,ccdskysb_galaxy_z,exptime_galaxy_g,exptime_galaxy_r,exptime_galaxy_z,meansky_galaxy_g,meansky_galaxy_r,meansky_galaxy_z,galdepth_galaxy_g,galdepth_galaxy_r,galdepth_galaxy_z
0,172191,0.509522,1.180820,0.787620,-0.558135,1.615369,4.563708,0.027940,617.485215,23.461968,...,19.328887,143.100496,139.892237,122.275265,0.199914,0.920791,2.869334,23.626332,22.942529,22.215109
1,174071,0.106983,0.750799,0.191747,-0.558135,1.406610,4.458577,0.018477,367.511248,23.441590,...,19.497361,128.317445,87.744524,106.219834,0.213351,0.564238,1.995409,23.598964,22.922293,22.342016
2,174554,0.106983,0.658651,0.083406,-0.116269,2.301381,6.055535,0.131243,459.871429,23.432264,...,19.849928,200.000000,124.000000,221.028571,0.126744,0.333307,3.754301,24.007800,23.363166,22.229356
3,175644,2.119678,2.317303,1.356409,0.325596,2.293624,5.733834,0.078246,302.710029,23.362907,...,19.853787,197.676991,82.831858,184.026549,0.133381,0.458713,3.285305,23.978089,22.985436,22.300106
4,176036,1.717139,1.887282,1.410579,-0.116269,1.553945,4.240670,0.013369,380.169409,23.332227,...,19.843108,154.074110,88.452018,85.079953,0.223669,0.568713,1.837093,23.634297,22.958253,22.356429


In [93]:
print(df_kitanidis.isna().sum())

pixel_id             0
lrg_density          0
elg_density          0
qso_density          0
stellar_density      0
airmass_galaxy       0
fwhm_galaxy          0
ebv_galaxy           0
ccdnphotom_galaxy    0
ccdskysb_galaxy_g    0
ccdskysb_galaxy_r    0
ccdskysb_galaxy_z    0
exptime_galaxy_g     0
exptime_galaxy_r     0
exptime_galaxy_z     0
meansky_galaxy_g     0
meansky_galaxy_r     0
meansky_galaxy_z     0
galdepth_galaxy_g    0
galdepth_galaxy_r    0
galdepth_galaxy_z    0
dtype: int64


#### Geometric Systematics Values

In [94]:
with open('../bricks_data/pixel2systematics_geometric.pickle', 'rb') as f:
    geometric_dict = pickle.load(f)
    f.close()

print(len(geometric_dict.keys()))


455672


In [97]:
# Clear the dict of all pixels where we do not have galaxy density values

pixels = df.pixel_id.tolist()
pixel_keys = list(geometric_dict.keys())
for pix in pixel_keys:
    if pix in pixels:
        continue
    geometric_dict.pop(pix)
print(len(geometric_dict.keys()))


2002


In [98]:
sys_list = []
for key in geometric_dict.keys():
    ind_sys_list = geometric_dict[key]
    ind_sys_list.append(key)
    sys_list.append(ind_sys_list)

df_sys_geo = pd.DataFrame(sys_list, columns=['airmass_galaxy',
'fwhm_galaxy',
'ebv_galaxy',
'ccdnphotom_galaxy',

'ccdskysb_galaxy_g',
'ccdskysb_galaxy_r',
'ccdskysb_galaxy_z',
'exptime_galaxy_g',
'exptime_galaxy_r',
'exptime_galaxy_z',
'meansky_galaxy_g',
'meansky_galaxy_r',
'meansky_galaxy_z',
'galdepth_galaxy_g',
'galdepth_galaxy_r',
'galdepth_galaxy_z', 'pixel_id'])



In [100]:
df_sys_geo.head()

,airmass_galaxy,fwhm_galaxy,ebv_galaxy,ccdnphotom_galaxy,ccdskysb_galaxy_g,ccdskysb_galaxy_r,ccdskysb_galaxy_z,exptime_galaxy_g,exptime_galaxy_r,exptime_galaxy_z,meansky_galaxy_g,meansky_galaxy_r,meansky_galaxy_z,galdepth_galaxy_g,galdepth_galaxy_r,galdepth_galaxy_z,pixel_id
0,1.204914,4.487709,0.021031,263.930656,0.369211,0.355531,0.312636,1.40625,1.40625,1.40625,2.882011e+06,8.985916e+06,1.687051e+08,0.362452,0.362690,0.338896,741107
1,1.068818,4.836110,0.010475,148.159223,0.369262,0.355497,0.312262,1.40625,1.40625,1.40625,1.933899e+06,3.522100e+06,3.096106e+08,0.368000,0.365229,0.336382,662895
2,1.075285,4.707209,0.009762,140.153964,0.369132,0.355331,0.312593,1.40625,1.40625,1.40625,2.392615e+06,6.166106e+06,3.290776e+08,0.364940,0.362549,0.335756,663887
3,1.182497,4.657945,0.009392,128.780124,0.369063,0.353477,0.314093,1.40625,1.40625,1.40625,9.302651e+05,1.120082e+07,1.377514e+08,0.367557,0.362241,0.337711,647690
4,1.102870,4.578874,0.015086,132.655989,0.368482,0.354034,0.316766,1.40625,1.40625,1.40625,5.450756e+06,1.090426e+07,1.230249e+08,0.355768,0.359193,0.338312,653815


In [101]:
df_geometric = df.merge(df_sys_geo, how='inner', on='pixel_id')
print(df_geometric.isna().sum())

pixel_id             0
lrg_density          0
elg_density          0
qso_density          0
stellar_density      0
airmass_galaxy       0
fwhm_galaxy          0
ebv_galaxy           0
ccdnphotom_galaxy    0
ccdskysb_galaxy_g    0
ccdskysb_galaxy_r    0
ccdskysb_galaxy_z    0
exptime_galaxy_g     0
exptime_galaxy_r     0
exptime_galaxy_z     0
meansky_galaxy_g     0
meansky_galaxy_r     0
meansky_galaxy_z     0
galdepth_galaxy_g    0
galdepth_galaxy_r    0
galdepth_galaxy_z    0
dtype: int64


,pixel_id,lrg_density,elg_density,qso_density,stellar_density,airmass_galaxy,fwhm_galaxy,ebv_galaxy,ccdnphotom_galaxy,ccdskysb_galaxy_g,...,ccdskysb_galaxy_z,exptime_galaxy_g,exptime_galaxy_r,exptime_galaxy_z,meansky_galaxy_g,meansky_galaxy_r,meansky_galaxy_z,galdepth_galaxy_g,galdepth_galaxy_r,galdepth_galaxy_z
0,166098,0.308252,0.075052,0.056321,0.325596,1.220610,3.898208,0.016500,1550.658093,0.364892,...,0.301166,1.515938,1.309821,1.509230,0.003598,0.011705,0.032974,0.366218,0.355911,0.348341
1,167123,0.408887,-0.416400,0.110491,-0.116269,1.219311,3.862342,0.016510,1587.507883,0.364887,...,0.301310,1.515938,1.309821,1.427000,0.003594,0.011709,0.036043,0.366503,0.356100,0.346743
2,171167,-0.496826,0.412925,-0.024934,-0.558135,1.655938,4.672048,0.028977,579.146330,0.366651,...,0.302868,2.300308,2.244602,1.939581,0.003221,0.015671,0.079584,0.368576,0.358520,0.344488
3,172190,-0.496826,-0.293537,-0.133275,-0.116269,1.658746,4.609840,0.028156,577.106054,0.366584,...,0.304636,2.370826,2.023661,1.915658,0.002980,0.016154,0.094865,0.369514,0.357950,0.342820
4,172191,0.509522,1.180820,0.787620,-0.558135,1.657476,4.666779,0.028076,585.977177,0.366563,...,0.303110,2.371346,2.025000,1.950375,0.002980,0.012420,0.047528,0.369459,0.358867,0.346243


In [103]:
print(df_kitanidis.shape)
print(df_geometric.shape)


(1151, 21)
(2002, 21)
